# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

## Assignment 4: Gradient Descent and Maximum Likelihood

In [3]:
require './assignment_lib'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

true

## Question 1.1 (5 points)

Let's implement a test function for the gradient descent optimizer, a 3D simple parabola. All gradient-optimized trainers are implemented as a objective function. The follow the same basic pattern:

```ruby
class MyGradientLearnableModel
  def func x, w
    #Returns the value of the objective function, 
    #  summing across all examples in x
  end
  def grad x, w
    #Returns a hash of derivative values for each variable in w,
    # gradient is summed across all examples in x
    dw = {"0" => (w["0"] - 1), "1" => (w["1"] - 2)}
  end
  def adjust w
    # Applies any problem-specific alterations to w
  end
end
```

Now, let's implement a Parabola objective function which does not depend on the data at all. It is defined as follows:

### $L(w) = \frac{1}{2}\left( \left(w_{0} - 1\right)^2 + \left(w_{0} - 2\right)^2 \right)$

Implement the ```func``` method for the loss function, $L(w)$. 

In [11]:
class ParabolaObjective
  def func x, w
    # BEGIN YOUR CODE
    return 0.5 * ((w["0"] - 1) ** 2 + (w["1"] - 2) ** 2) 
  end
  def adjust w
  end
end

:adjust

In [12]:
### TESTS ###
t1 = ParabolaObjective.new
assert_in_delta(0.0, t1.func([], {"0" => 1.0, "1" => 2.0}), 1e-3)
assert_in_delta(0.5, t1.func([], {"0" => 1.0, "1" => 1.0}), 1e-3)
assert_in_delta(0.5, t1.func([], {"0" => 1.0, "1" => 3.0}), 1e-3)
assert_in_delta(2.5, t1.func([], {"0" => 3.0, "1" => 1.0}), 1e-3)


## Question 1.2 (5 Points)

Implement the gradient function for $L(w)$. It evaluates the gradient for the value of $x$ for each dimension of $w$. In this simple case, $L(w)$ does not depend on $x$.

In [13]:
class ParabolaObjective
  def grad x, w
    # BEGIN YOUR CODE
    return {"0" => w["0"] - 1, "1" => w["1"] - 2};
    #END YOUR CODE
  end
end

:grad

In [14]:
### TESTS ###
t2 = ParabolaObjective.new
w2_1 = t2.grad([], {"0" => 3.0, "1" => 7.0})
assert_in_delta(2.0, w2_1["0"], 1e-3)
assert_in_delta(5.0, w2_1["1"], 1e-3)

w2_2 = t2.grad([], {"0" => -3.0, "1" => -7.0})
assert_in_delta(-4.0, w2_2["0"], 1e-3)
assert_in_delta(-9.0, w2_2["1"], 1e-3)


## Question 2.1 (1 Point)


Implement gradient descent for any objective function class. Your function must provide a callback which we will use to monitor its performance and possibly to halt execution. A simple example illustrating the callback is as follows:

```ruby
def gradient_descent_example dataset, w, obj, learning_rate, tol, max_iter, &block
    iter = 0
    until converged(w_last, w) do
        w_last = w
        loss = calc_loss(w)
        w = update(w)
        w = adjust(w)
        break unless yield w, iter, loss
        iter += 1
    end
    
    return w
end

```

There are three main parts to the algorithm above:
1. Convergence is based on the norm of the difference of the weight vectors
1. The norm can be calculated as the dot product of two vectors: $||w|| = w \cdot w$
1. Once the loss and gradient functions are calculated, we will update the values of each weight

In this first part, implement the dot product

In [25]:
#Implement the error function given a weight vector, w
def dot x, w
  # BEGIN YOUR CODE
    sum = 0.0
    
    if !(x.empty? or w.empty?)
      x.each do |k, v|
          if w.has_key?(k)
              sum += v * w[k]
          end
      end
    end
    
    return sum
  #END YOUR CODE
end


:dot

In [26]:
### TEST ###
assert_in_delta 6.0, dot({"a" => 2.0}, {"a" => 3.0}), 1e-6
assert_in_delta 6.0, dot({"a" => 2.0}, {"a" => 3.0, "b" => 4.0}), 1e-6
assert_equal 0.0, dot({}, {})
assert_equal 0.0, dot({"a" => 1.0}, {"b" => 1.0})

## Question 2.2 (1 Point)
Implement the L2 norm for a vector, represented by a hash. Hint: use ```dot```.

In [31]:
def norm w
  # BEGIN YOUR CODE
  return 0.5 * Math.sqrt(dot(w, w))
  #END YOUR CODE
end

:norm

In [32]:
### TEST ###
assert_in_delta 1.0, norm({"a" => 1.41421, "b" => 1.41421}), 1e-2
assert_in_delta 1.0, norm({"a" => -1.41421, "b" => 1.41421}), 1e-2
assert_in_delta 0.0, norm({}), 1e-2

## Question 2.3 (3 points)
Implement a function that updates a weight vector, ```w```, given a gradient vector, ```dw```, and learning rate, ```lr```.  Do not change the original weight vector. Hint: use ```clone```.

In [37]:
def update_weights(w, dw, lr)
  # BEGIN YOUR CODE
  wTemp = w.clone
  wTemp.each do |k, v|
    wTemp[k] -= dw[k] * lr
  end 
  #END YOUR CODE
end

:update_weights

In [38]:
### TEST ###
assert_in_delta 1.5, update_weights({"a" => 1.0}, {"a" => -0.25}, 2.0)["a"], 1e-2
assert_in_delta 2.5, update_weights({"a" => 1.0, "b" => 3.0}, {"a" => -0.25, "b" => 0.25}, 2.0)["b"], 1e-2

## Question 2.4 (15 Points)

Now, put all these functions together to implement gradient descent. 

In [ ]:
def gradient_descent dataset, w, obj, learning_rate, tol, max_iter, &block
  # BEGIN YOUR CODE
  iter = 0
    until converged(w_last, w) do
        w_last = w
        loss = calc_loss(w)
        w = update(w)
        w = adjust(w)
        break unless yield w, iter, loss
        iter += 1
    end

    return w
  #END YOUR CODE
  return w
end

In [ ]:
### TESTS ###

t4 = ParabolaObjective.new
t4_w_init = {"0" => 3.0, "1" => 7.0}
t4_w_goal = {"0" => 1.0, "1" => 2.0}
t4_data = {"data" => []}

t4_loss = 1.0
t4_w = nil
gradient_descent t4_data, t4_w_init, t4, 0.1, 0.001, 100 do |w, iter, loss|
  t4_loss = loss
  t4_w = w
end

assert_in_delta 0.01, t4_loss, 1e-2
assert_in_delta 1.0, t4_w["0"], 1e-1
assert_in_delta 2.0, t4_w["1"], 1e-1

In [ ]:
### TEST ###
t4 = ParabolaObjective.new
t4_w_init = {"0" => 3.0, "1" => 7.0}
t4_data = {"data" => []}

t4_total_loss = 0.0
t4_iterations = []
t4_losses = []
gradient_descent t4_data, t4_w_init, t4, 0.1, 0.001, 100 do |w, iter, loss|
  t4_total_loss += loss
  t4_iterations << iter
  t4_losses << t4_total_loss / iter + 1.0
end

assert_true t4_iterations.size > 30
assert_true t4_losses[-1] < 3
assert_true t4_losses[-1] > 0
assert_true t4_iterations[-1] > 30

In [ ]:
### Plot the cumulative loss per iteration
Daru::DataFrame.new({x: t4_iterations, y: t4_losses}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
  plot.x_label "Iteration"
  plot.y_label "Cumulative Loss"
end

## Question 3.1 (5 Points)

Let's learn the parameter of a Bernoulli distribution using the method of maximum likelihood. Consider the following dataset in which we are tossing a biased coin with probability $\mu$ of returning a success (1). There is an analytical solution for the parameter $\mu$ given a dataset of successes and trials. Compute this analytical solution. 

Here the ```label``` field is either 0 or 1. 

In [ ]:
coin_data = coin_dataset(1000)

def q31_binomial_parameter(coin_data)
  # BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
end

In [ ]:
### TESTS ###

t31_coin_data = coin_dataset(10000)
assert_in_delta 0.77, q31_binomial_parameter(t31_coin_data), 5e-2

## Question 3.2 (5 Points)

Now, let's use the maximum likelihood function and gradient descent to find the same parameter value. Define the objective function for a binomial distribution for multiple samples. Remember that the ```label``` is the target value and every example has only one feature, ```bias```. Learn the weight for the ```bias``` feature should converge to $w_{bias} = \mu$.

In this first step, calculate the log likelihood function for the binomial distribution of $n$ examples.

In [ ]:
class BinomialModel
  def func dataset, w
    # BEGIN YOUR CODE
    raise NotImplementedError.new()
    #END YOUR CODE
  end
end

In [ ]:
### TEST ###
t32_model = BinomialModel.new
t32_t1 = {"features" => {"bias" => 1.0}, "label" => 0.0}
t32_t2 = {"features" => {"bias" => 1.0}, "label" => 1.0}

assert_in_delta 1.469677, t32_model.func([t32_t1], {"bias" => 0.77}), 1e-3
assert_in_delta 0.261365, t32_model.func([t32_t2], {"bias" => 0.77}), 1e-3
assert_in_delta 1.731041, t32_model.func([t32_t1, t32_t2], {"bias" => 0.77}), 1e-3


## Question 3.3 (5 Points)

Calculate the gradient of the binomial log likelihood function over $n$ examples.

In [ ]:
class BinomialModel
  def grad dataset, w
    # BEGIN YOUR CODE
    raise NotImplementedError.new()
    #END YOUR CODE
    return g
  end
  
  ## Adjusts the parameter to be within the allowable range
  def adjust w
    w["bias"] = [[0.001, w["bias"]].max, 0.999].min
  end
end

In [ ]:
### TEST ###
t32_model = BinomialModel.new
t32_t1 = {"features" => {"bias" => 1.0}, "label" => 0.0}
t32_t2 = {"features" => {"bias" => 1.0}, "label" => 1.0}

assert_in_delta 4.347826, t32_model.grad([t32_t1], {"bias" => 0.77})["bias"], 1e-3
assert_in_delta -1.29870, t32_model.grad([t32_t2], {"bias" => 0.77})["bias"], 1e-3
assert_in_delta 3.049124, t32_model.grad([t32_t1, t32_t2], {"bias" => 0.77})["bias"], 1e-3

## Question 3.4 (5 Points)

Putting the objective function to work, use gradient descent to find the parameter for the binomial distribution. You get to set the learning rate. Return the learning rate you have obtained which works well. You may have to try a few until you get it right.

Note that, while capable of returning the same value, this method reads the data many more times than the analytical solution.

In [ ]:
def q34_learning_rate
  # BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
end

In [ ]:
### TEST ###
t34 = BinomialModel.new
t34_w_init = {"bias" => rand}
t34_data = coin_dataset(10000)

t34_learning_rate = q34_learning_rate()

t34_total_loss = 0.0
t34_iterations = []
t34_losses = []
t34_w = t34_w_init
gradient_descent t34_data, t34_w_init, t34, t34_learning_rate, 0.001, 100 do |w, iter, loss|
  puts [iter, w, t34_losses[-1]].join("\t") if iter % 10 == 1
  t34_total_loss += loss
  t34_iterations << iter
  t34_losses << t34_total_loss / iter.to_f
  t34_w = w
end


assert_true(t34_losses[-1] < 8000)
assert_true(t34_losses[-1] > 0)
assert_true(t34_iterations[-1] > 30)
assert_in_delta 0.77, t34_w["bias"], 5e-2

### Plot the cumulative loss per iteration
Daru::DataFrame.new({x: t34_iterations, y: t34_losses}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
  plot.x_label "Iteration"
  plot.y_label "Cumulative Loss"
end

## Question 4.1 (10 Points)

The maximum likelihood method above reads the data multiple times and can benefit from prior knowledge in the form of a prior distribution for the parameter, $\mu$. Using the Beta distribution as the conjugate prior, implement the likelihood function and its gradient. Now we are learning three parameters altogether: $w_{bias} = \mu$, $\alpha$, $\beta$.

First, let's compute the closed form estimator for $\mu$ with a fixed $\alpha$ and $\beta$.

In [ ]:
def q41_closed_form_beta_binomial(coin_data, alpha, beta)
  # BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
end

In [ ]:
### TEST ###
t41_coin_data = coin_dataset(10000)
assert_in_delta 0.77, q41_closed_form_beta_binomial(t41_coin_data, 7743, 10000 - 7743), 1e-2

## Question 4.2 (10 points)

Implement the negative log likelihood function for the beta + binomial values. Checkout this [Wikipedia](https://en.wikipedia.org/wiki/Beta_distribution#Maximum_likelihood) definition of the likelihood function. Remember we are interested in he negative log likelihood.

A special function is needed ```GSL::Sf::lnbeta(a, b)```.

In [ ]:
class BetaBinomialModel
  def func dataset, w
    # BEGIN YOUR CODE
    raise NotImplementedError.new()
    #END YOUR CODE
  end
  def adjust w
    w["bias"] = [[0.001, w["bias"]].max, 0.999].min
    w["beta"] = [0.0001, w["beta"]].max
  end
end

In [ ]:
### TESTS ###

t42 = BetaBinomialModel.new
srand 777 #seed random number generator
t42_data = coin_dataset(1000)["data"]
t42_w = Hash.new {|h,k| h[k] = 0.1}
t42_w["alpha"] = 7.0
t42_w["beta"] = 3.0

assert_in_delta 10373.126026759332, t42.func(t42_data, t42_w), 1e-2

## Question 4.3 (10 points)

Implement the negative log likelihood gradient for all the parameters. Checkout this [Wikipedia](https://en.wikipedia.org/wiki/Beta_distribution#Maximum_likelihood) definition of the likelihood function. Remember we are interested in he negative log likelihood.

A special function is needed ```GSL::Sf::psi(a + b)```.

In [ ]:
class BetaBinomialModel
  def grad dataset, w
    # BEGIN YOUR CODE
    raise NotImplementedError.new()
    #END YOUR CODE
  end
end

In [ ]:
### TESTS ###

t43 = BetaBinomialModel.new
srand 777 #seed random number generator
t43_data = coin_dataset(1000)["data"]
t43_w = Hash.new {|h,k| h[k] = 0.1}
t43_w["alpha"] = 7.0
t43_w["beta"] = 3.0

t43_grad = t43.grad(t43_data, t43_w)

assert_in_delta -65622.22222222318, t43_grad["bias"], 1e2
assert_in_delta 1923.6168390257724, t43_grad["alpha"], 1e2
assert_in_delta -1223.6077383104214, t43_grad["beta"], 1e2


## Question 4.4 (20 points)

Run the gradient descent by selecting the initial weights and learning rate. Try a few values. 

In [ ]:
def q44_weights_and_learning_rate
  # BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
  return [w, lr]
end

In [ ]:
### TEST ###
t44 = BetaBinomialModel.new
t44_data = coin_dataset(10000)

t44_w_init, t44_learning_rate = q44_weights_and_learning_rate()

t44_total_loss = 0.0
t44_iterations = []
t44_losses = []
t44_w = t34_w_init
gradient_descent t44_data, t44_w_init, t44, t44_learning_rate, 0.001, 100 do |w, iter, loss|
  puts [iter, w, t44_losses[-1]].join("\t") if iter % 10 == 1
  t44_total_loss += loss
  t44_iterations << iter
  t44_losses << t34_total_loss / iter.to_f
  t44_w = w
end


assert_true(t44_losses[-1] < 8000)
assert_true(t44_losses[-1] > 0)
assert_true(t44_iterations[-1] > 30)
assert_in_delta 0.77, t44_w["bias"], 5e-2

### Plot the cumulative loss per iteration
Daru::DataFrame.new({x: t44_iterations, y: t44_losses}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
  plot.x_label "Iteration"
  plot.y_label "Cumulative Loss"
end